<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-mcp/blob/main/openai-mcp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using OpenAI Agents with a MCP Server

In [ ]:
# Install required packages

!pip install --upgrade pip
!pip install fastmcp openai nest_asyncio


This code sets up and runs a FastMCP server that acts as a weather service.

It uses fastmcp to create an agent that can interact with the National Weather Service (NWS) API. It defines several Tools like get_forecast, get_alerts, and health_check which perform specific actions. The server is then launched in a background thread to make these tools and resources accessible via HTTP, specifically configured for notebook compatibility.

The MCP server is based upon code found at https://github.com/lxchst/weather-server-python/blob/main/src/weather/server.py

In [ ]:
# The MCP Server

from fastmcp import FastMCP
import nest_asyncio
import threading
import time
from typing import Any
import httpx

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

nest_asyncio.apply()

mcp = FastMCP(
    name="WeatherServer",
    instructions="Provides an up to date weather forecast and alerts for any location and also includes a health check."
)

async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {"User-Agent": USER_AGENT, "Accept": "application/geo+json"}
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None


def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
        Event: {props.get("event", "Unknown")}
        Area: {props.get("areaDesc", "Unknown")}
        Severity: {props.get("severity", "Unknown")}
        Description: {props.get("description", "No description available")}
        Instructions: {props.get("instruction", "No specific instructions provided")}
        """

@mcp.tool
def greet(name: str) -> str:
    """Greet a person by their name.

    Args:
        name: The name of the person to greet.
    """
    return f"Hello, {name}!"


@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)


@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
        {period["name"]}:
        Temperature: {period["temperature"]}°{period["temperatureUnit"]}
        Wind: {period["windSpeed"]} {period["windDirection"]}
        Forecast: {period["detailedForecast"]}
        """
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)


@mcp.tool()
def health_check():
    """Returns the health status of the server."""
    return {"status": "ok"}


# Define the function to run the server
def run_server():
    # Use transport="streamable-http" for compatibility with notebooks/Colab
    print("🚀 Starting FastMCP server in background thread...")
    mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")


This code connects to a local FastMCP server to retrieve and then call its available tools. It defines several asynchronous functions, each designed to interact with a specific tool on the server (like greet, get_alerts, get_forecast, health_check, and list_tools), and then it executes these functions to demonstrate how to use the server's capabilities.

In [ ]:
# First try to connect to the MCP server.

import asyncio
from fastmcp import Client

client = Client("http://localhost:8000/mcp")

async def call_greet(name: str):
    async with client:
        result = await client.call_tool("greet", {"name": name})
        print(result)


async def call_get_alerts(name: str):
    async with client:
        result = await client.call_tool("get_alerts", {"state": name})
        print(result)


async def call_get_forecast(latitude: float, longitude: float):
    async with client:
        result = await client.call_tool("get_forecast", {"latitude": latitude, "longitude": longitude})
        print(result)


async def call_health_check():
    async with client:
        result = await client.call_tool("health_check")
        print(result)


async def call_list_tools():
    async with client:
        result = await client.list_tools()
        for tool in result:
            print(tool)

asyncio.run(call_greet("Norm"))
asyncio.run(call_get_alerts("WA"))
asyncio.run(call_health_check())
asyncio.run(call_get_forecast(47.7179, -116.9516))
asyncio.run(call_list_tools())


This code demonstrates how to integrate a FastMCP server with an OpenAI agent. It first connects to the local FastMCP server to retrieve the tools it exposes (like weather forecast and alerts). These tools are then used to create an OpenAI agent. Finally, the agent is run with a user input, and it leverages the provided FastMCP tools to find the answer, in this case, asking about the weather in Seattle.

In [ ]:
from fastmcp import Client
from openai import OpenAI
import asyncio
import os
from google.colab import userdata

# Initialize API key
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

MCP_SERVER_URL = "http://localhost:8000/mcp"

async def main():
    # Connect to your MCP server
    mcp_client = Client(MCP_SERVER_URL)

    # Fetch FastMCP tool definitions within the context manager
    mcp_tools = []
    async with mcp_client:
        mcp_tools = await mcp_client.list_tools()

    # Convert to OpenAI Assistant tool format
    agent_tools = []
    for tool in mcp_tools:
        # OpenAI function tools require details to be nested under a 'function' key
        agent_tools.append({
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description if tool.description else "",
                "parameters": tool.inputSchema
            }
        })

    # Create the assistant (formerly 'agent')
    openai_client = OpenAI()
    assistant = openai_client.beta.assistants.create(
        model="gpt-4.1",
        name="MCP Assistant",
        instructions="Use the provided tools when helpful.",
        tools=agent_tools
    )

    print("Assistant created:", assistant.id)

    # Create a thread
    thread = openai_client.beta.threads.create()

    # Add a message to the thread
    message = openai_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="What is the weather in Seattle?"
    )

    # Create and poll the run
    run = openai_client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Handle tool calls if needed (simplified for this example)
    if run.status == 'requires_action':
        # In a real application, you would process run.required_action.submit_tool_outputs
        # here, call the MCP server tools, and then submit the outputs back to OpenAI.
        print("Run requires action. Tool calls need to be handled.")

    # Get the response
    if run.status == 'completed':
        messages = openai_client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Print the last message from the assistant
        if messages.data and messages.data[0].role == "assistant":
            print(messages.data[0].content[0].text.value)
        else:
            print("No response from assistant or unexpected message format.")
    else:
        print(f"Run ended with status: {run.status}")

asyncio.run(main())